# Practical Steps with FuzzManager

In this chapter we will perform basic steps around FuzzManager, including crash submission and triage as well as coverage measurement tasks.

**Prerequisites**

* This chapter builds directly upon the contents of the [last chapter](Fuzzing_in_the_Large.ipynb), so we recommend you read that before proceeding.
* This chapter requires you to be reading the book using the **Docker image**, as it uses a pre-installed demo instance of FuzzManager. Starting the Docker image is really straightforward...
* C/C++ knowledge is helpful if you would like to understand the crash examples. It is not required though to do so and you can perfectly learn the concepts of this chapter without fully understanding why these programs behave the way they do.

## Server Setup

This Docker image already has a FuzzManager Demo Server running in the background for you. It will be used during the following exercises. If you are prompted to login at any point in time, use the following credentials:

<center>**Username: demo**</center>
<center>**Password: demo**</center>

At the end of the chapter, you will find a link to setup instructions, in case you would like to run your own instance for future fuzzing work.

## Basic Crash Processing

To get started with basic steps in crash processing, let's take a look at *simply-buggy*, an example repository containing trivial C++ programs for illustration purposes.

In [ ]:
!git clone https://github.com/choller/simply-buggy
!(cd simply-buggy && make)

The make command compiles our target program, including our first target, the *simple-crash* example. Alongside the program, there is also a configuration file generated. Let's take a look at that file and the source code:

In [ ]:
!cat simply-buggy/simple-crash.cpp

In [ ]:
!cat simply-buggy/simple-crash.fuzzmanagerconf

As you can see, the source code is fairly simple: A forced crash by writing to a (near)-NULL pointer. The configuration file generated for the the binary also contains some straightforward information, like the revision of the program and other metadata that is required or at least useful later on when submitting things.

Running the program shows us a crash trace as expected:

In [ ]:
!simply-buggy/simple-crash

Now, what we would actually like to do is to run this binary from Python instead, detect that it crashed, collect the trace and submit it to the server. Let's start with a simple script that would just run the program we give it and detect the presence of the ASan trace:

In [ ]:
import subprocess

cmd = ["simply-buggy/simple-crash"]

result = subprocess.run(cmd, stderr=subprocess.PIPE)
stderr = result.stderr.decode().splitlines()
crashed = False

for line in stderr:
    if "ERROR: AddressSanitizer" in line:
        crashed = True
        break

if crashed:
    print("Yay, we crashed!")
else:
    print("Move along, nothing to see...")

Nice, we can now run the binary and detect that it crashed. But how do we send this information to the server now? Let's make a few modifications...

In [ ]:
import subprocess

from Collector.Collector import Collector
from FTB.ProgramConfiguration import ProgramConfiguration
from FTB.Signatures.CrashInfo import CrashInfo

# Instantiate the collector instance, this will be our entry point
# for talking to the server.
collector = Collector()

cmd = ["simply-buggy/simple-crash"]

result = subprocess.run(cmd, stderr=subprocess.PIPE)
stderr = result.stderr.decode().splitlines()
crashed = False

for line in stderr:
    if "ERROR: AddressSanitizer" in line:
        crashed = True
        break

if crashed:
    print("Yay, we crashed, processing...")
    
    # This reads the simple-crash.fuzzmanagerconf file
    configuration = ProgramConfiguration.fromBinary(cmd[0])
    
    # This reads and parses our ASan trace into a more generic format,
    # returning us a generic "CrashInfo" object that we can inspect
    # and/or submit to the server.
    crashInfo = CrashInfo.fromRawCrashData([], stderr, configuration)
    
    # Submit the crash
    collector.submit(crashInfo)
    
    print("Crash submitted!")
else:
    print("Move along, nothing to see...")

We now submitted something to our local FuzzManager demo instance. If you go to http://127.0.0.1:8000/crashmanager/crashes/ you should see your crash. Now click on the crash to inspect the submitted data. Then click the orange *Create* button to create a bucket for this crash. A *crash signature* will be proposed to you for matching this and future crashes of the same type, accept it by clicking *Save*. You will be redirected to the newly created bucket, which shows you the size (how many crashes it holds), it's bug report status (buckets can be linked to bugs in an external bug tracker like Bugzilla) and many other useful information. If you click on the *Signatures* entry in the top menu, you should also see your newly created entry.

Buckets and their signatures are a central concept in FuzzManager. If you receive a lot of crash reports from various sources, bucketing allows you to easily group crashes and filter duplicates. The flexible signature system starts out with an initially proposed fine-grained signature, but it can be adjusted as needed to capture variations of the same bug and make tracking easier. In the next example, we will look at a more complex example that reads data from a file and creates multiple crash signatures.

In [ ]:
!cat simply-buggy/out-of-bounds.cpp

This program looks way more elaborate compared to the last one, but don't worry, it isn't really doing a whole lot. The code in the `main` function simply reads a file provided on the command line and puts its contents into a buffer that is passed to `validateAndPerformAction`. That function pulls out two bytes of the buffer (`action` and `count`) and considers the rest `data`. Depending on the value of `action`, it then calls either `printFirst` or `printLast`, which prints either the first or the last `count` bytes of `data`. Sounds pointless, and yes, it is. The whole idea of this program is that the security check (that `count` is not larger than the length of `data`) is missing in `validateAndPerformAction` but that the illegal access happens later in either of the two print functions. Hence, we would expect this program to generate at least two (slightly) different crash signatures. Let's try it out with very simple fuzzing based on the last Python script:

In [ ]:
import os
import random
import subprocess
import tempfile
import sys

from Collector.Collector import Collector
from FTB.ProgramConfiguration import ProgramConfiguration
from FTB.Signatures.CrashInfo import CrashInfo

# Instantiate the collector instance, this will be our entry point
# for talking to the server.
collector = Collector()

cmd = ["simply-buggy/out-of-bounds"]

crash_count = 0

for itnum in range(0,100):
    rand_len = random.randint(1, 1024)
    rand_data = bytearray(os.urandom(rand_len))
    
    (fd, current_file) = tempfile.mkstemp(prefix="fuzztest")
    os.write(fd, rand_data)
    os.close(fd)
    
    current_cmd = []
    current_cmd.extend(cmd)
    current_cmd.append(current_file)
    
    result = subprocess.run(current_cmd, stderr=subprocess.PIPE)
    stderr = result.stderr.decode().splitlines()
    crashed = False

    for line in stderr:
        if "ERROR: AddressSanitizer" in line:
            crashed = True
            break

    if crashed:
        sys.stdout.write("C")

        # This reads the simple-crash.fuzzmanagerconf file
        configuration = ProgramConfiguration.fromBinary(cmd[0])

        # This reads and parses our ASan trace into a more generic format,
        # returning us a generic "CrashInfo" object that we can inspect
        # and/or submit to the server.
        crashInfo = CrashInfo.fromRawCrashData([], stderr, configuration)

        # Submit the crash
        collector.submit(crashInfo, testCase = current_file)
        
        crash_count += 1
    else:
        sys.stdout.write(".")
    
    os.remove(current_file)

print("")
print("Done, submitted %s crashes." % crash_count)

If you run this script, you will see it's progress and notice that it produces quite a few crashes. And indeed, if you visit [FuzzManager](http://127.0.0.1:8000/crashmanager/crashes/), you will notice a variety of crashes that have accumulated. Pick the first crash and create a bucket for it, like you did the last time. After saving, you will notice that not all of your crashes went into the bucket. The reason is that our program created several different stacks that are somewhat similar but not exactly identical. This is a common problem when fuzzing real world applications. Fortunately, there is an easy way to deal with this. While on the bucket page, hit the *Optimize* button for the bucket. FuzzManager will then automatically propose you to change your signature. Accept the change by hitting *Edit with Changes* and then *Save*. Repeat these steps until all crashes are part of the bucket. After 3 to 4 iterations, your signature will likely look like this:

```{
  "symptoms": [
    {
      "type": "output",
      "src": "stderr",
      "value": "/ERROR: AddressSanitizer: heap-buffer-overflow/"
    },
    {
      "type": "stackFrames",
      "functionNames": [
        "?",
        "?",
        "?",
        "validateAndPerformAction",
        "main",
        "__libc_start_main",
        "_start"
      ]
    },
    {
      "type": "crashAddress",
      "address": "> 0xFF"
    }
  ]
}```

As you can see in the *stackFrames* symptom, the `validateAndPerformAction` stack frame is still present because it is common in all crashes (in fact, this is where the bug lives). But the lower stack parts have been removed because they vary across the set of submitted crashes.

The *Optimize* function is designed to automate this process as much as possible: It attempts to broaden the signature by fitting it to untriaged crashes and then checks if the modified signature would touch other existing buckets. This works with the assumption that other buckets are indeed other bugs, i.e. if you had created two buckets from your crashes first, optimizing wouldn't work anymore. Also, if the existing bucket data is sparse and you have a lot of untriaged crashes, the algorithm could propose changes that include crashes of different bugs in the same bucket. There is no way to fully automatically detect and prevent this, hence the process is semi-automated and requires you to review all proposed changes.